In [ ]:
# Install latest version sklearn (0.23)
!pip install -U scikit-learn

     |████████████████████████████████| 22.2MB 5.4MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
# Imports
import pandas as pd
import numpy as np
import os
import shutil
from collections import Counter
import matplotlib.pyplot as plt
import time
import gc
import sys
import random
import pickle

from scipy.stats import iqr
from sklearn import preprocessing

import datetime
import logging

# Deep learning
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
import tensorflow.keras.models as M

from IPython.display import display
# Set pandas options
pd.options.display.max_rows = 2000

In [ ]:
# Global variables 
THR_E = 100 # interaction threshold of exercises (E) for user
BATCH_SIZE = 128
TRAIN_FRACTION = 0.95

AUTO = tf.data.experimental.AUTOTUNE
EPOCHS = 160
N_SELECT_PER_EPOCH = 100000 # Random select N samples for each epoch from train/val set
VAL_EVERY_N_EPOCHS = 5
PRINT_EVERY_N_BATCHES = 50 

SAVE_DICTS = True

In [ ]:
# SEED THE EXPERIMENTS
np.random.seed(18)

In [ ]:
# Experiment date and number!
date = datetime.datetime.today().strftime("%d-%b")
experiment = "-riiid-1"
OUTPUT_FOLDER = date + experiment 
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

In [ ]:
# Selected features
selections = ["E", "r", "etc", "ltg", "at", "ra"]
assert((selections[0] == "E") & (selections[1]=="r"))

In [ ]:
# Choose model setting
ENC_EMB, ENC_DENSE = [0, 4], [] 
DEC_EMB, DEC_DENSE = [1, 2, 3, 5], []

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

FOLDER_FEATHER = "/content/drive/My Drive/kaggle-riiid/feather-files"
PREPROCESS_FILE = "/content/drive/My\ Drive/Colab\ Notebooks/demo-riiid-preprocessing.ipynb"
MODEL_FILE = "/content/drive/My\ Drive/Colab\ Notebooks/demo-riiid-transformer.ipynb" # https://stackoverflow.com/questions/57464810/how-to-run-a-jupyter-notebook-with-space-in-relative-path-from-another-notebook
# MODEL_FILE = "/content/drive/My\ Drive/Colab\ Notebooks/riiid-functional-transformer.ipynb" # https://stackoverflow.com/questions/57464810/how-to-run-a-jupyter-notebook-with-space-in-relative-path-from-another-notebook


Mounted at /content/drive


In [ ]:
# Add functions to preprocess data
%run $PREPROCESS_FILE

In [ ]:
# Create logger
logging = create_logging(OUTPUT_FOLDER)

In [ ]:
%%time
# Read all dataframes from feather files and print out
train = read_df_print(os.path.join(FOLDER_FEATHER, "train.feather")) 
questions = read_df_print(os.path.join(FOLDER_FEATHER, "questions.feather"))
lectures = read_df_print(os.path.join(FOLDER_FEATHER, "lectures.feather"))

(101230332, 10)


,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,False,1,3,1,NaN,None
1,1,56943,115,5716,False,2,2,1,37000.0,False
2,2,118363,115,128,False,0,0,1,55000.0,False


(13523, 5)


,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92


(418, 4)


,lecture_id,tag,part,type_of
0,89,24584,5,concept
1,100,22243,1,concept
2,185,7035,6,concept


CPU times: user 1.76 s, sys: 2.62 s, total: 4.38 s
Wall time: 22.5 s


In [ ]:
train["content_type_id"].sum()

1959032

In [ ]:
# train.loc[train["content_type_id"]==1, "content_id"].value_counts()

1959032

In [ ]:
# Preprocess for lectures
if ("v" in selections) or ("l" in selections):
  lectures["lecture_id"] = pd.Categorical(lectures["lecture_id"])
  lectures["lec_id"] = lectures["lecture_id"].cat.codes
  lec_map = dict(zip(lectures["lecture_id"], lectures["lec_id"]))

  train.loc[train["content_type_id"]==1, "content_id"] = train.loc[train["content_type_id"]==1, "content_id"].map(lec_map)
# train["v"] = train["content_type_id"].shift(1).fillna(0).astype(np.int8)
# train["vc"] = train.groupby("user_id")["v"].cumsum()

In [ ]:
# We don't need the lectures now!
train = train.loc[train["content_type_id"]==0]

27-12 15:27 numexpr.utils INFO     NumExpr defaulting to 4 threads.


In [ ]:
# Save space by deleting some columns
del train["user_answer"]

In [ ]:
N_questions, N_parts = questions["question_id"].nunique(), questions["part"].nunique()
N_response, N_task, N_lag, N_et, N_groups, N_attempt, N_avg, N_ltg = 2, 2, 1440, 301, 10, 8, 101, 400 # N_avg = 100 for regular?

In [ ]:
# Seq len of users [sorted same as df]. USED FOR TRAINING
seq_len_dict = dict(train["user_id"].value_counts())
users = train["user_id"].unique()
seq_lens = [seq_len_dict[user] for user in users]

In [ ]:
# Train/valid split
train_len = int(len(users)*TRAIN_FRACTION)
val_len = len(users) - train_len
train_users, val_users = users[:train_len], users[train_len:]

# PROBS of sequence
seq_len_train, seq_len_val = seq_lens[:train_len], seq_lens[train_len:]
PROBS_TRAIN = seq_len_train/np.sum(seq_len_train)
PROBS_VAL = seq_len_val/np.sum(seq_len_val)

In [ ]:
len(train_users), len(val_users)

(373973, 19683)

In [ ]:
train["prior_question_had_explanation"].isnull().sum() # Less than users - why?

392506

## Add all inputs/outputs

In [ ]:
p_lists, lt_lists, et_lists, tag_lists, task_lists, quantile_transformer_et, lt_cat, quantile_transformer_lt, et_cat, ltc, etc, ex, etg, ra, ca, at, r_dup, ltg, v_lists  = [], [], [], [], [], [], [], [], [], [],[], [], [], [], [], [], [], [], []

In [ ]:
# Lag time grouped
if "ltg" in selections:
    %time ltg, N_ltg = return_ltg(N_ltg)

CPU times: user 39.8 s, sys: 111 ms, total: 40 s
Wall time: 39.3 s


In [ ]:
# Add exercises
%time E_lists = return_E()

CPU times: user 25.4 s, sys: 1.3 s, total: 26.7 s
Wall time: 26.7 s


In [ ]:
# Add results, with or without start token
%time r_lists = return_r(add_start_token=True)

CPU times: user 26.5 s, sys: 442 ms, total: 26.9 s
Wall time: 26.9 s


In [ ]:
if "v" in selections:
    %time v_lists = return_v()

In [ ]:
if "r_dup" in selections:
   %time r_dup = r_lists.apply(lambda x: x[:-1])

In [ ]:
if "ca" in selections:
    %time ca = return_ca()

In [ ]:
if "at" in selections:
    %time at = return_attempt()

CPU times: user 59.3 s, sys: 7.23 s, total: 1min 6s
Wall time: 1min 6s


In [ ]:
if "ra" in selections:
    %time ra = return_r_avg(add_start_token=True, N_avg = N_avg)

CPU times: user 40.7 s, sys: 829 ms, total: 41.5 s
Wall time: 40.5 s


In [ ]:
if "ex" in selections:
    %time ex = return_ex()

In [ ]:
bins = list(range(30)) + [32, 35, 40, 48, 60, 80, 120, 299, 300]
if "etg" in selections:
    %time etg = return_etg(bins)
    # %time etg = return_etg(N_groups)


In [ ]:
if "ltc" in selections:
    %time ltc = return_ltc()

In [ ]:
if "lt" in selections:
    %time lt_lists, quantile_transformer_lt = return_lt()

In [ ]:
if "etc" in selections:
    %time etc = return_etc(zero_task=True)

CPU times: user 39.5 s, sys: 746 ms, total: 40.2 s
Wall time: 39.9 s


In [ ]:
if "p" in selections:
    %time p_lists = return_p()

In [ ]:
if "et" in selections:
    %time et_lists, quantile_transformer_et = return_et()

In [ ]:
if "tag" in selections:
    %time tag_lists = return_N_highest_tags() # TODO: specify N tags

In [ ]:
if "task" in selections:
    %time task_lists = return_task_binary()

In [ ]:
feature_mapping = {"E": E_lists, "r": r_lists, "p": p_lists, "et": et_lists, "lt": lt_lists, "tag": tag_lists, "task": task_lists, "et_std": et_lists, "ltc": ltc, "etc": etc, "ex": ex, "etg": etg, "ra": ra, "at": at, "ca": ca, "r_dup": r_dup, "v": v_lists, "ltg": ltg}
type_mapping = {"E": tf.int32, "r": tf.int32, "p": tf.int32, "et": tf.float32, "et_std": tf.float32, "lt": tf.float32, "tag": tf.float32, "task": tf.float32, "ltc":tf.int32, "etc": tf.int32, "ex": tf.int32, "etg": tf.int32, "ra": tf.int32, "at": tf.int32, "ca": tf.int32, "r_dup": tf.int32, "v": tf.int32, "ltg": tf.int32}
pad_mapping = {"E": N_questions, "r": N_response, "p": 0, "et": 0.5, "et_std": 0.0, "lt": 0.5, "tag": 2.0, "task": float(N_task), "ltc": N_lag+1, "etc": N_et+1, "ex": 2, "etg": len(bins)+1, "ra": N_avg+1, "at": N_attempt, "ca": N_avg+1, "r_dup": N_response, "v": N_response, "ltg": N_ltg+1}
vocab_mapping = {"E": N_questions+1, "r": N_response+2, "et":None, "p": N_parts+1, "lt": None, "tag": 3.0, "task": float(N_task+1), "et_std": float(300), "ltc": N_lag+2, "etc": N_et+2, "ex": 4, "etg": len(bins)+2, "ra": N_avg+2, "at": N_attempt+1, "ca": N_avg+2, "r_dup": N_response+2, "v": N_response+1, "ltg": N_ltg+2}
pad_shapes = {"E": [THR_E], "r": [THR_E+1], "p": [THR_E], "et": [THR_E], "et_std": [THR_E],  "lt": [THR_E], "tag": [THR_E], "task": [THR_E], "ltc": [THR_E], "etc": [THR_E], "ex": [THR_E], "etg": [THR_E], "ra": [THR_E], "at": [THR_E], "ca": [THR_E], "r_dup": [THR_E], "v": [THR_E], "ltg": [THR_E]}

In [ ]:
def save_as_dict(df, filename):
    seq_dict = df.to_dict()
    with open(os.path.join(OUTPUT_FOLDER_DICT, filename), 'wb') as handle:
      pickle.dump(seq_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
if SAVE_DICTS:
  OUTPUT_FOLDER_DICT = f"seq-dict-{THR_E}"
  os.makedirs(OUTPUT_FOLDER_DICT, exist_ok=True)

  for feature in selections:
      feat_df = feature_mapping[feature]
      feat_df = feat_df.apply(lambda x: x[-THR_E:]) # Take last THR_E in history 
      filename = f"{feature}.pickle"
      save_as_dict(feat_df, filename)
  shutil.move(OUTPUT_FOLDER_DICT, OUTPUT_FOLDER)

In [ ]:
# # Add start token!
# # Answers
# r_lists = r_lists.apply(lambda x: [3] + x) # Start token = 3
# feature_mapping["r"] = r_lists

In [ ]:
# # Running answers
# if "ra" in selections:
#     ra = ra.apply(lambda x: [N_avg] + x[:-1]) # Start token = 101
#     feature_mapping["ra"] = ra

In [ ]:
vocab_sizes = [vocab_mapping[select] for select in selections]
feature_lists = [feature_mapping[select] for select in selections]

In [ ]:
PADDING_VALUES = tuple((pad_mapping[select] for select in selections))
OUTPUT_TYPES = tuple((type_mapping[select] for select in selections))
PADDED_SHAPES = tuple((pad_shapes[select] for select in selections))

In [ ]:
feature_lists

[user_id
 115           [5692, 5716, 128, 7860, 7922, 156, 51, 50, 789...
 124           [7900, 7876, 175, 1278, 2064, 2063, 2065, 3364...
 2746          [5273, 758, 5976, 236, 404, 382, 405, 873, 531...
 5382          [5000, 3944, 217, 5844, 5965, 4990, 5235, 6050...
 8623          [3915, 4750, 6456, 3968, 6104, 5738, 6435, 549...
                                     ...                        
 2147470770    [7900, 7876, 175, 1278, 2064, 2065, 2063, 3363...
 2147470777    [7900, 7876, 175, 1278, 2065, 2064, 2063, 3365...
 2147481750    [4137, 1270, 9261, 8201, 367, 378, 214, 6071, ...
 2147482216    [3748, 4765, 5474, 9261, 4665, 5987, 6666, 561...
 2147482888    [6147, 4792, 5738, 6102, 4748, 7956, 6435, 928...
 Name: content_id, Length: 393656, dtype: object, user_id
 115           [3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, ...
 124           [3, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, ...
 2746          [3, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, ...
 5382          [3, 1, 0

In [ ]:
try:
  del train
except Exception as e:
  print("train already deleted")
gc.collect(), gc.collect()

train already deleted


(7740, 0)

# Create train/val datasets

In [ ]:
# Lists of numpy arrays
train_list = [features.values[:train_len] for features in feature_lists]
val_list = [features.values[train_len:] for features in feature_lists]

In [ ]:
def select_window_size(*x):
    series_len = len(x[0]) # Length of series
    if series_len <= THR_E: # Just return the sequence!
        return x
    else: # Random select from sequence
        max_select = series_len - THR_E
        random_select = tf.random.uniform(shape=(), minval=0, maxval=max_select, dtype=tf.int32)
        x = [i[random_select:random_select+THR_E] for i in x]
        x = tuple(x)
        return x

In [ ]:
train_list

[array([list([5692, 5716, 128, 7860, 7922, 156, 51, 50, 7896, 7863, 152, 104, 108, 7900, 7901, 7971, 25, 183, 7926, 7927, 4, 7984, 45, 185, 55, 7876, 6, 172, 7898, 175, 100, 7859, 57, 7948, 151, 167, 7897, 7882, 7962, 1278, 2065, 2064, 2063, 3363, 3365, 3364]),
        list([7900, 7876, 175, 1278, 2064, 2063, 2065, 3364, 3365, 3363, 2948, 2947, 2946, 2595, 2593, 2594, 4492, 4120, 4696, 6116, 6173, 6370, 6909, 6910, 6908, 6911, 7218, 7216, 7217, 7219]),
        list([5273, 758, 5976, 236, 404, 382, 405, 873, 531, 775, 294, 714, 297, 297, 775, 1295, 10684, 1014, 484]),
        ...,
        list([7900, 7876, 175, 1278, 2064, 2063, 2065, 3364, 3365, 3363, 2947, 2948, 2946, 2593, 2595, 2594, 4492, 4120, 4696, 6116, 6173, 6370, 6878, 6880, 6879, 6877, 7219, 7218, 7217, 7216, 6106, 4755, 9313, 3586, 4476, 6432, 5845, 9094, 6191, 5437]),
        list([7900, 7876, 175, 1278, 2064, 2063, 2065, 3364, 3363, 3365, 2946, 2948, 2947, 2594, 2595, 2593, 4492, 4120, 4696, 6116, 6173, 6370, 6879, 6880, 6

In [ ]:
def create_train_dataset(N_training_per_epoch, probs_train):
  index = np.random.choice(train_len, N_training_per_epoch, replace=True, p = probs_train) # random indexing (similar to shuffling)
  tr_list = [features[index] for features in train_list] # Selecting by index 
  train_ds = (tf.data.Dataset
                  .from_generator(lambda: iter(zip(*tr_list)), output_types=OUTPUT_TYPES)
                  .map(select_window_size)
                  .padded_batch(batch_size = BATCH_SIZE, padded_shapes = PADDED_SHAPES, padding_values = PADDING_VALUES)
                  .prefetch(AUTO)
  )
  return train_ds

In [ ]:
def create_val_dataset(N_valid_per_epoch, probs_val):
  index = np.random.choice(val_len, N_valid_per_epoch, replace=True, p = probs_val) # random indexing (similar to shuffling)
  vl_list = [features[index] for features in val_list] # Selecting by index 
  val_ds = (tf.data.Dataset
                  .from_generator(lambda: iter(zip(*vl_list)), output_types=OUTPUT_TYPES)
                  .map(select_window_size)
                  .padded_batch(batch_size = BATCH_SIZE, padded_shapes = PADDED_SHAPES, padding_values = PADDING_VALUES)
                  .cache()
                  .prefetch(AUTO)
  )
  return val_ds

In [ ]:
val_dataset = create_val_dataset(N_SELECT_PER_EPOCH, PROBS_VAL)
a = iter(val_dataset) # Small check

In [ ]:
next(a)

(<tf.Tensor: shape=(128, 100), dtype=int32, numpy=
 array([[   41, 10617,     5, ...,  5252,  4410,  9468],
        [ 4758,  5689,  5241, ..., 10540, 10584,     9],
        [ 3379,  3380,  3378, ...,  8868,  6276,  4416],
        ...,
        [ 4190,  4020,  6145, ...,  2182,  2184,  2064],
        [ 7898,   112,   457, ...,  3510,  3512,  3511],
        [ 4476,  4633,  9484, ...,  1071,   846,   923]], dtype=int32)>,
 <tf.Tensor: shape=(128, 101), dtype=int32, numpy=
 array([[1, 1, 1, ..., 1, 1, 2],
        [0, 0, 1, ..., 1, 1, 2],
        [1, 1, 0, ..., 1, 1, 2],
        ...,
        [0, 0, 1, ..., 1, 0, 2],
        [1, 0, 1, ..., 0, 1, 2],
        [1, 1, 1, ..., 1, 1, 2]], dtype=int32)>,
 <tf.Tensor: shape=(128, 100), dtype=int32, numpy=
 array([[37, 20, 22, ..., 27, 14, 25],
        [31, 21, 21, ..., 15, 20, 19],
        [47,  0,  0, ..., 70, 30, 33],
        ...,
        [29, 20, 42, ...,  0,  0, 35],
        [15, 24, 29, ..., 20,  0,  0],
        [ 2,  2,  9, ..., 23, 11, 21]], d

## Get model and set hyperparameters

In [ ]:
%run $MODEL_FILE

In [ ]:
shutil.copy('/content/drive/My Drive/Colab Notebooks/demo-riiid-transformer.ipynb', OUTPUT_FOLDER)

'27-Dec-riiid-1/demo-riiid-transformer.ipynb'

In [ ]:
# Hyperparameters main model
config = {"features": selections, "n_features": len(selections), "vocab_sizes": vocab_sizes, 
          "enc_emb": ENC_EMB, "enc_dense": ENC_DENSE, "dec_emb": DEC_EMB, "dec_dense": DEC_DENSE, 
          "window_size": THR_E, "enc_num_layers": 2, "dec_num_layers": 2, "d_model": 256, 
          "dff": 512, "num_heads": 8, "dropout_rate": 0.1, "pos_encoding": True,
          "padding_values": PADDING_VALUES, "output_types": OUTPUT_TYPES, "padded_shapes": PADDED_SHAPES, 
          "quantile_transformer_et": quantile_transformer_et,  "quantile_transformer_lt": quantile_transformer_lt}

with open(os.path.join(OUTPUT_FOLDER, "config.pickle"), 'wb') as handle:
  pickle.dump(config, handle, protocol=pickle.HIGHEST_PROTOCOL)

logging.info('config: %s', config)

27-12 15:35 root         INFO     config: {'features': ['E', 'r', 'etc', 'ltg', 'at', 'ra'], 'n_features': 6, 'vocab_sizes': [13524, 4, 303, 175, 9, 103], 'enc_emb': [0, 4], 'enc_dense': [], 'dec_emb': [1, 2, 3, 5], 'dec_dense': [], 'window_size': 100, 'enc_num_layers': 2, 'dec_num_layers': 2, 'd_model': 256, 'dff': 512, 'num_heads': 8, 'dropout_rate': 0.1, 'pos_encoding': True, 'padding_values': (13523, 2, 302, 174, 8, 102), 'output_types': (tf.int32, tf.int32, tf.int32, tf.int32, tf.int32, tf.int32), 'padded_shapes': ([100], [101], [100], [100], [100], [100]), 'quantile_transformer_et': [], 'quantile_transformer_lt': []}


In [ ]:
transformer = create_model_separate_input(config)
# tf.keras.utils.plot_model(transformer, os.path.join(OUTPUT_FOLDER, "model_plot.png"), show_shapes=True)

## Optimizer

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()
    
    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps
  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)
    
    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [ ]:
learning_rate = CustomSchedule(config["d_model"])
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.999, 
                                      epsilon=1e-9)

## Loss and metrics

Since the target sequences are padded, it is important to apply a padding mask when calculating the loss.

In [ ]:
# Loss and metric
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

accuracy_object = tf.keras.metrics.sparse_categorical_accuracy

train_auc= tf.keras.metrics.AUC()
val_auc= tf.keras.metrics.AUC()

In [ ]:
def loss_function(real, pred): # batch_size x seq_size x 1 vs  batch_size x seq_size x 3
    mask = tf.math.logical_not(tf.math.equal(tf.squeeze(real), N_response)) # batch_size x seq_size
    loss_ = loss_object(real, pred) # batch_size x seq_size
    mask = tf.cast(mask, dtype=loss_.dtype)
    
    loss_ *= mask 
    loss_ = tf.reduce_sum(loss_)/tf.reduce_sum(mask) #loss becomes one value! (from all batches)
    return loss_

In [ ]:
def metric_function(real, pred, auc_object): # batch_size x seq_size x 1 vs  batch_size x seq_size x 3
    # Predict accuracy
    mask = tf.math.logical_not(tf.math.equal(tf.squeeze(real), N_response)) # batch_size x seq_size
    accuracy = accuracy_object(real, pred) # batch_size x seq_size
    mask = tf.cast(mask, dtype=accuracy.dtype)
    
    accuracy *= mask
    accuracy = 100*tf.reduce_sum(accuracy)/tf.reduce_sum(mask)
    
    # A work-around to predict AUC => is it stable?
    pred = tf.nn.softmax(pred)
    pred = pred[:,:,1] # pred that answer is correct
    real = tf.keras.backend.flatten(real)
    pred = tf.keras.backend.flatten(pred)
    
    idxs = tf.math.logical_not(tf.math.equal(real, N_response))
    real = real[idxs]
    pred = pred[idxs]
    auc = auc_object(real, pred)
    return accuracy

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.Mean(name='train_accuracy')

val_loss = tf.keras.metrics.Mean(name='val_loss')
val_accuracy = tf.keras.metrics.Mean(name='val_accuracy')

## Training and checkpointing

Create the checkpoint path and the checkpoint manager. This will be used to save checkpoints every `n` epochs.

In [ ]:
checkpoint_path = os.path.join(OUTPUT_FOLDER, "checkpoints/")
ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

# https://stackoverflow.com/questions/62919208/how-to-restore-a-specific-checkpoint-in-tensorflow2-to-implement-early-stopping

# # if a checkpoint exists, restore the latest checkpoint.
# if ckpt_manager.latest_checkpoint:
#   ckpt.restore(ckpt_manager.latest_checkpoint)
#   print ('Latest checkpoint restored!!')

In [ ]:
@tf.function()
def train_step(tar_real, *inputs):
    with tf.GradientTape() as tape: 
        predictions = transformer(inputs, training=True)
        tar_real = tf.expand_dims(tar_real, -1) # IMPORTANT! DOESN'T WORK WITHOUT IT. ALWAYS GIVES ERROR INCOMPATIBLE SHAPE. E.G. (32,169) vs. (32,169,3)
        
        loss = loss_function(tar_real, predictions)
        accuracy = metric_function(tar_real, predictions, auc_object = train_auc)
    
    gradients = tape.gradient(loss, transformer.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

    train_loss(loss)
    train_accuracy(accuracy)

@tf.function()
def val_step(tar_real, *inputs):
    predictions = transformer(inputs, training=False)
    tar_real = tf.expand_dims(tar_real, -1) # IMPORTANT! DOESN'T WORK WITHOUT IT. ALWAYS GIVES ERROR INCOMPATIBLE SHAPE. E.G. (32,169) vs. (32,169,3)
    
    loss = loss_function(tar_real, predictions)
    accuracy = metric_function(tar_real, predictions, auc_object = val_auc)

    val_loss(loss)
    val_accuracy(accuracy)

In [ ]:
best_auc = 0
for epoch in range(EPOCHS):
    train_dataset = create_train_dataset(N_SELECT_PER_EPOCH, PROBS_TRAIN) # create train for each epoch
    start = time.time()

    train_loss.reset_states()
    train_accuracy.reset_states()
    train_auc.reset_states()
    
    val_loss.reset_states()
    val_accuracy.reset_states()
    val_auc.reset_states()

    # Train
    for (batch, (features)) in enumerate(train_dataset):
        E, tar, rest = features[0], features[1], features[2:] 
        tar_inp = tar[:, :-1]
        tar_real = tar[:, 1:]

        inputs = [E, tar_inp] + list(rest)        
        train_step(tar_real, inputs)
        
        if batch % PRINT_EVERY_N_BATCHES == 0:
            print ('Epoch {} TRAIN Batch {} Loss {:.4f} Accuracy {:.4f} AUC {:.4f}'.format(
              epoch + 1, batch, train_loss.result(), train_accuracy.result(), train_auc.result()))

    logging.info('Epoch TRAIN {} Loss {:.4f} Accuracy {:.4f} AUC {:.4f}'.format(epoch + 1, 
                                                train_loss.result(), 
                                                train_accuracy.result(),
                                                train_auc.result()
                                                ))
    
    # Validate and save model
    if (epoch + 1) % VAL_EVERY_N_EPOCHS == 0: 
      for (batch, (features)) in enumerate(val_dataset):
          E, tar, rest = features[0], features[1], features[2:] 
          tar_inp = tar[:, :-1]
          tar_real = tar[:, 1:]

          inputs = [E, tar_inp] + list(rest)          
          val_step(tar_real, inputs)

          if batch % PRINT_EVERY_N_BATCHES == 0:
              print ('Epoch {} VAL Batch {} Loss {:.4f} Accuracy {:.4f} AUC {:.4f}'.format(
                epoch + 1, batch, val_loss.result(), val_accuracy.result(), val_auc.result()))
      
      logging.info('Epoch VAL {} Loss {:.4f} Accuracy {:.4f} AUC {:.4f}'.format(epoch + 1, 
                                          val_loss.result(), 
                                          val_accuracy.result(),
                                          val_auc.result()
                                          ))    
       
      if val_auc.result() > best_auc:
          best_auc = val_auc.result()
          ckpt_save_path = ckpt_manager.save()
          logging.info('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                            ckpt_save_path))
        
    print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

Epoch 1 TRAIN Batch 0 Loss 0.9299 Accuracy 63.7076 AUC 0.5083
Epoch 1 TRAIN Batch 50 Loss 0.7663 Accuracy 61.8310 AUC 0.5053
Epoch 1 TRAIN Batch 100 Loss 0.7218 Accuracy 62.6180 AUC 0.5039
Epoch 1 TRAIN Batch 150 Loss 0.7012 Accuracy 63.3085 AUC 0.5056
Epoch 1 TRAIN Batch 200 Loss 0.6885 Accuracy 63.8791 AUC 0.5076
Epoch 1 TRAIN Batch 250 Loss 0.6795 Accuracy 64.3067 AUC 0.5109
Epoch 1 TRAIN Batch 300 Loss 0.6724 Accuracy 64.6960 AUC 0.5150
Epoch 1 TRAIN Batch 350 Loss 0.6661 Accuracy 65.0000 AUC 0.5232
Epoch 1 TRAIN Batch 400 Loss 0.6606 Accuracy 65.2510 AUC 0.5326
Epoch 1 TRAIN Batch 450 Loss 0.6561 Accuracy 65.4574 AUC 0.5410
Epoch 1 TRAIN Batch 500 Loss 0.6521 Accuracy 65.6529 AUC 0.5487
Epoch 1 TRAIN Batch 550 Loss 0.6491 Accuracy 65.7869 AUC 0.5545
Epoch 1 TRAIN Batch 600 Loss 0.6462 Accuracy 65.9305 AUC 0.5605
Epoch 1 TRAIN Batch 650 Loss 0.6436 Accuracy 66.0594 AUC 0.5658
Epoch 1 TRAIN Batch 700 Loss 0.6411 Accuracy 66.1915 AUC 0.5705
Epoch 1 TRAIN Batch 750 Loss 0.6390 Accurac

27-12 15:38 root         INFO     Epoch TRAIN 1 Loss 0.6379 Accuracy 66.3547 AUC 0.5767


Time taken for 1 epoch: 169.53773474693298 secs

Epoch 2 TRAIN Batch 0 Loss 0.6087 Accuracy 68.1979 AUC 0.6357
Epoch 2 TRAIN Batch 50 Loss 0.6092 Accuracy 67.7044 AUC 0.6340
Epoch 2 TRAIN Batch 100 Loss 0.6077 Accuracy 67.7520 AUC 0.6393
Epoch 2 TRAIN Batch 150 Loss 0.6070 Accuracy 67.7711 AUC 0.6405
Epoch 2 TRAIN Batch 200 Loss 0.6065 Accuracy 67.7716 AUC 0.6420
Epoch 2 TRAIN Batch 250 Loss 0.6059 Accuracy 67.7867 AUC 0.6431
Epoch 2 TRAIN Batch 300 Loss 0.6051 Accuracy 67.8252 AUC 0.6449
Epoch 2 TRAIN Batch 350 Loss 0.6048 Accuracy 67.8075 AUC 0.6460
Epoch 2 TRAIN Batch 400 Loss 0.6043 Accuracy 67.8277 AUC 0.6471
Epoch 2 TRAIN Batch 450 Loss 0.6040 Accuracy 67.8404 AUC 0.6475
Epoch 2 TRAIN Batch 500 Loss 0.6037 Accuracy 67.8544 AUC 0.6483
Epoch 2 TRAIN Batch 550 Loss 0.6033 Accuracy 67.8724 AUC 0.6492
Epoch 2 TRAIN Batch 600 Loss 0.6027 Accuracy 67.9133 AUC 0.6501
Epoch 2 TRAIN Batch 650 Loss 0.6015 Accuracy 67.9882 AUC 0.6527
Epoch 2 TRAIN Batch 700 Loss 0.5981 Accuracy 68.2474 AUC 0

27-12 15:41 root         INFO     Epoch TRAIN 2 Loss 0.5922 Accuracy 68.6873 AUC 0.6709


Time taken for 1 epoch: 164.4950349330902 secs

Epoch 3 TRAIN Batch 0 Loss 0.5387 Accuracy 72.8545 AUC 0.7614
Epoch 3 TRAIN Batch 50 Loss 0.5360 Accuracy 72.8684 AUC 0.7596
Epoch 3 TRAIN Batch 100 Loss 0.5348 Accuracy 72.8971 AUC 0.7610
Epoch 3 TRAIN Batch 150 Loss 0.5338 Accuracy 72.9727 AUC 0.7622
Epoch 3 TRAIN Batch 200 Loss 0.5334 Accuracy 72.9866 AUC 0.7626
Epoch 3 TRAIN Batch 250 Loss 0.5333 Accuracy 72.9909 AUC 0.7629
Epoch 3 TRAIN Batch 300 Loss 0.5328 Accuracy 73.0282 AUC 0.7632
Epoch 3 TRAIN Batch 350 Loss 0.5325 Accuracy 73.0505 AUC 0.7635
Epoch 3 TRAIN Batch 400 Loss 0.5326 Accuracy 73.0374 AUC 0.7635
Epoch 3 TRAIN Batch 450 Loss 0.5323 Accuracy 73.0571 AUC 0.7638
Epoch 3 TRAIN Batch 500 Loss 0.5321 Accuracy 73.0729 AUC 0.7641
Epoch 3 TRAIN Batch 550 Loss 0.5318 Accuracy 73.0929 AUC 0.7644
Epoch 3 TRAIN Batch 600 Loss 0.5318 Accuracy 73.0858 AUC 0.7646
Epoch 3 TRAIN Batch 650 Loss 0.5316 Accuracy 73.1011 AUC 0.7648
Epoch 3 TRAIN Batch 700 Loss 0.5313 Accuracy 73.1216 AUC 0.

...

27-12 22:20 root         INFO     Epoch TRAIN 150 Loss 0.4923 Accuracy 75.3102 AUC 0.8061


Epoch 150 VAL Batch 0 Loss 0.4943 Accuracy 75.4376 AUC 0.7987
Epoch 150 VAL Batch 50 Loss 0.4969 Accuracy 75.1462 AUC 0.8015
Epoch 150 VAL Batch 100 Loss 0.4953 Accuracy 75.2445 AUC 0.8025
Epoch 150 VAL Batch 150 Loss 0.4952 Accuracy 75.2417 AUC 0.8026
Epoch 150 VAL Batch 200 Loss 0.4951 Accuracy 75.2460 AUC 0.8028
Epoch 150 VAL Batch 250 Loss 0.4955 Accuracy 75.2152 AUC 0.8028
Epoch 150 VAL Batch 300 Loss 0.4960 Accuracy 75.1777 AUC 0.8025
Epoch 150 VAL Batch 350 Loss 0.4962 Accuracy 75.1524 AUC 0.8025
Epoch 150 VAL Batch 400 Loss 0.4964 Accuracy 75.1454 AUC 0.8023
Epoch 150 VAL Batch 450 Loss 0.4964 Accuracy 75.1484 AUC 0.8023
Epoch 150 VAL Batch 500 Loss 0.4964 Accuracy 75.1467 AUC 0.8023
Epoch 150 VAL Batch 550 Loss 0.4965 Accuracy 75.1467 AUC 0.8022
Epoch 150 VAL Batch 600 Loss 0.4963 Accuracy 75.1574 AUC 0.8023
Epoch 150 VAL Batch 650 Loss 0.4965 Accuracy 75.1467 AUC 0.8022
Epoch 150 VAL Batch 700 Loss 0.4964 Accuracy 75.1532 AUC 0.8023
Epoch 150 VAL Batch 750 Loss 0.4965 Accurac

27-12 22:21 root         INFO     Epoch VAL 150 Loss 0.4966 Accuracy 75.1379 AUC 0.8023


Time taken for 1 epoch: 183.86889052391052 secs

Epoch 151 TRAIN Batch 0 Loss 0.5057 Accuracy 74.7874 AUC 0.7914


KeyboardInterrupt: ignored

## Summary

In this tutorial, you learned about positional encoding, multi-head attention, the importance of masking and how to create a transformer.

Try using a different dataset to train the transformer. You can also create the base transformer or transformer XL by changing the hyperparameters above. You can also use the layers defined here to create [BERT](https://arxiv.org/abs/1810.04805) and train state of the art models. Futhermore, you can implement beam search to get better predictions.

# Older code

In [ ]:
# # From https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
# class DataGenerator(keras.utils.Sequence):
#     'Generates data for Keras'
#     def __init__(self, df, shuffle=True):
#         'Initialization'
        
#         self.df = df
#         self.users = self.df["user_id"].nunique()
#         self.batch_size = BATCH_SIZE
#         self.shuffle = shuffle
#         self.on_epoch_end()

#     def __len__(self):
#         'Denotes the number of batches per epoch'
#         return int(np.floor(len(self.list_IDs) / self.batch_size))

#     def __getitem__(self, index):
#         'Generate one batch of data'
#         # Generate indexes of the batch
#         indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

#         # Find list of IDs
#         list_IDs_temp = [self.list_IDs[k] for k in indexes]

#         # Generate data
#         X, y = self.__data_generation(list_IDs_temp)

#         return X, y

#     def on_epoch_end(self):
#         'Updates indexes after each epoch'
#         self.indexes = np.arange(len(self.list_IDs))
#         if self.shuffle == True:
#             np.random.shuffle(self.indexes)

#     def __data_generation(self, list_IDs_temp):
#         'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
#         # Initialization
#         X = np.empty((self.batch_size, *self.dim, self.n_channels))
#         y = np.empty((self.batch_size), dtype=int)

#         # Generate data
#         for i, ID in enumerate(list_IDs_temp):
#             # Store sample
#             X[i,] = np.load('data/' + ID + '.npy')

#             # Store class
#             y[i] = self.labels[ID]

#         return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

In [ ]:
# def save_dict(seq_dict, filename):
#     with open(os.path.join(OUTPUT_FOLDER, filename), 'wb') as handle:
#       pickle.dump(seq_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# # Save last N dict to memory
# E_dict = E_lists.apply(lambda x: x[-200:]).to_dict()
# r_dict = r_lists.apply(lambda x: x[-200:]).to_dict()
# et_dict = et_lists.apply(lambda x: x[-200:]).to_dict()

In [ ]:
# save_dict(E_dict, "E.pkl")
# save_dict(r_dict, "r.pkl")
# save_dict(et_dict, "et.pkl")